In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U peft
# !pip install -q -U flash_attn --no-build-isolation
!pip uninstall flash_attn -y
!pip install -q -U datasets

!pip install -q -U evaluate
!pip install -q -U tokenizers

!pip install -q diffusers --upgrade
!pip install -q invisible_watermark accelerate safetensors

!pip install -q rouge
!pip install -q rouge_score

!pip install -q bert_score

!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import evaluate
from pprint import pprint
from transformers import AutoConfig

import datasets
import bitsandbytes as bnb
import torch
import random
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from PIL import Image
import requests

import re
import numpy as np
from scipy.special import softmax

import torch
import transformers
from datasets import Dataset, load_dataset

from transformers import pipeline, BitsAndBytesConfig
from transformers import CLIPProcessor, TFCLIPModel

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# For style classifier model (also for evaluating the seq2seq model output)
from transformers import TrainingArguments, Trainer

In [ ]:
# Set memory fraction for L4 (typically 22-24GB available)
torch.cuda.set_per_process_memory_fraction(0.95)
torch.cuda.empty_cache()

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Modify this path to the appropriate location in your Drive
train_data = pd.read_excel('drive/MyDrive/266/project/train_data.xlsx')
val_data = pd.read_excel('drive/MyDrive/266/project/val_data.xlsx')
test_data = pd.read_excel('drive/MyDrive/266/project/test_data.xlsx')

In [ ]:
def preprocess_text(df):
  """Convert Excel data to text pairs format"""
  text_pairs = []
  for _, row in df.iterrows():
      line_1 = str(row['line1']).lower()
      line_2 = str(row['line2']).lower()
      text_pairs.append((line_1, line_2))
  return text_pairs

In [ ]:
#Let's create some splits
train_pairs = preprocess_text(train_data)
val_pairs = preprocess_text(val_data)
test_pairs = preprocess_text(test_data)

# Final splits
splits = {
    'train': train_pairs,
    'val': val_pairs,
    'test': test_pairs
}

print(f"\nDataset sizes:")
print(f"{len(splits['train'])} training pairs")
print(f"{len(splits['val'])} validation pairs")
print(f"{len(splits['test'])} test pairs")

# Look at some examples
print("Training examples:")
for _ in range(5):
    print(random.choice(train_pairs))


Dataset sizes:
60000 training pairs
15000 validation pairs
15000 test pairs
Training examples:
("today they gon' read my nigga verdict", 'today i might post a throwback thursday')
('{1st verse}', "i'm a superhero, like lou ferrigno")
('see me on the mic man, everything i drop is ill', 'tryna find a key, you can call it glockenspiel')
("i had to sacrifice to be here, that's how i'm with you tonight", 'i had to run down with my big glock 9 to feed my appetite')
("dropped a couple mil' in the crib like a mall, homie, what's up?", 'yeah baby, show you all the finer things')


In [ ]:
# Load the pre-trained model and tokenizer

from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure quantization BEFORE loading the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Use 4-bit quantization for maximum memory savings
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation (faster + stable)
    bnb_4bit_use_double_quant=True,         # Double quantization for better accuracy
    bnb_4bit_quant_type="nf4"               # Normalized Float 4-bit quantization
)

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2',
#                                         quantization_config=quantization_config,
#                                         device_map="auto",
#                                         torch_dtype=torch.bfloat16)

# Add padding token (GPT-2 doesn't have one by default)
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# model.resize_token_embeddings(len(tokenizer))

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Configure LoRA for efficient training with quantized models
lora_config = LoraConfig(
    r=16,                    # Rank of adaptation
    lora_alpha=32,           # LoRA scaling parameter
    target_modules=["c_attn", "c_proj", "c_fc"],  # GPT-2 specific modules
    lora_dropout=0.1,        # LoRA dropout
    bias="none",             # Don't adapt bias parameters
    task_type="CAUSAL_LM"    # Task type for causal language modeling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Check trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"Trainable params: {trainable_params:,}")
    print(f"All params: {all_param:,}")
    print(f"Trainable %: {100 * trainable_params / all_param:.2f}%")

print_trainable_parameters(model)

# Verify quantization worked
if hasattr(model, 'hf_device_map'):
    print(f"Device map: {model.hf_device_map}")

# Check GPU memory usage
print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

# Verify model is quantized
print(f"Model dtype: {next(model.parameters()).dtype}")
print(f"Model device: {next(model.parameters()).device}")

Trainable params: 2,359,296
All params: 84,331,776
Trainable %: 2.80%
Device map: {'': 0}
GPU memory allocated: 0.38 GB
GPU memory reserved: 0.55 GB
Model dtype: torch.float32
Model device: cuda:0


In [ ]:
def prepare_song_data(splits, tokenizer, model, max_length=50):
    """Prepare text pairs for GPT-2 training"""

    resize_needed = False

    # Ensure we have a proper padding token that's different from EOS
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<PAD>'})
        resize_needed = True
        print(f"Added padding token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")

    if '<SEP>' not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'sep_token': '<SEP>'})
        resize_needed = True
        print(f"Added separator token: '{tokenizer.sep_token}'")

    # Resize model embeddings if we added tokens
    if resize_needed:
        model.resize_token_embeddings(len(tokenizer))
        print(f"Resized model embeddings to {len(tokenizer)} tokens")

    def tokenize_function(text_pairs):
        # Add task prefix to match your evaluation format
        task_prefix = "Given this rap line, generate the next line: "

        # Format: "task_prefix + line1 + <SEP> + line2 + <EOS> + padding"
        # inputs = [f"{task_prefix}{line1} <SEP> {line2}{tokenizer.eos_token}" for line1, line2 in text_pairs]
        inputs = []
        for line1, line2 in text_pairs:
            sequence = f"{task_prefix}{line1} {tokenizer.sep_token} {line2}{tokenizer.eos_token}"
            inputs.append(sequence)

        # Tokenize
        model_inputs = tokenizer(
            inputs,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )

        # Verify token usage
        sample_ids = model_inputs['input_ids'][0]
        if tokenizer.pad_token_id in sample_ids:
            pad_count = (sample_ids == tokenizer.pad_token_id).sum().item()
            print(f"✅ Using {pad_count} padding tokens in first sample")

        if tokenizer.sep_token_id in sample_ids:
            print(f"✅ Separator token found in sequence")

        if tokenizer.eos_token_id in sample_ids:
            print(f"✅ EOS token found in sequence")

        # For causal LM, labels are the same as input_ids
        model_inputs["labels"] = model_inputs["input_ids"].clone()
        model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100

        return model_inputs

    # Tokenize datasets
    train_tokenized = tokenize_function(splits['train'])
    val_tokenized = tokenize_function(splits['val'])

    # Convert to HuggingFace Dataset format for easier handling
    train_dataset = Dataset.from_dict({
        'input_ids': train_tokenized['input_ids'],
        'attention_mask': train_tokenized['attention_mask'],
        'labels': train_tokenized['labels']
    })

    val_dataset = Dataset.from_dict({
        'input_ids': val_tokenized['input_ids'],
        'attention_mask': val_tokenized['attention_mask'],
        'labels': val_tokenized['labels']
    })

    return train_dataset, val_dataset

# Prepare your data
train_dataset, val_dataset = prepare_song_data(splits, tokenizer, model)

✅ Using 14 padding tokens in first sample
✅ Separator token found in sequence
✅ EOS token found in sequence
✅ Using 17 padding tokens in first sample
✅ Separator token found in sequence
✅ EOS token found in sequence


In [ ]:
print(f"Added padding token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
print(f"EOS token: '{tokenizer.eos_token}' (ID: {tokenizer.convert_tokens_to_ids('<SEP>')})")

def verify_sequence_format(dataset, idx=0):
    """Verify that sequences follow the exact format you specified"""

    sample = dataset[idx]
    input_ids = sample['input_ids']
    labels = sample['labels']

    # Decode the full sequence (including special tokens)
    full_sequence = tokenizer.decode(input_ids, skip_special_tokens=False)

    print("🔍 Sequence Format Verification:")
    print(f"Full sequence: {full_sequence}")
    print(f"Input IDs: {input_ids}")
    print(f"Labels: {labels}")
    print()

    # Check for each component in order
    components = [
        ("Task prefix", "Given this rap line, generate the next line:"),
        ("Separator token", "<SEP>"),
        ("EOS token", tokenizer.eos_token),
        ("Padding token", tokenizer.pad_token)
    ]

    for name, token in components:
        if token in full_sequence:
            print(f"✅ {name} found: '{token}'")
        else:
            print(f"❌ {name} missing: '{token}'")

# Verify the format
verify_sequence_format(train_dataset, 0)

Added padding token: '<|endoftext|>' (ID: 50256)
EOS token: '<|endoftext|>' (ID: 50257)
🔍 Sequence Format Verification:
Full sequence: Given this rap line, generate the next line: my midwest slabbers - ya dig? <SEP> ay! yup yup, this is for my east coast slangers<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
Input IDs: [15056, 428, 4095, 1627, 11, 7716, 262, 1306, 1627, 25, 616, 3095, 7038, 38677, 1213, 532, 21349, 3100, 30, 220, 50257, 38762, 0, 331, 929, 331, 929, 11, 428, 318, 329, 616, 7627, 7051, 1017, 6606, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]
Labels: [15056, 428, 4095, 1627, 11, 7716, 262, 1306, 1627, 25, 616, 3095, 7038, 38677, 1213, 532, 21349, 3100, 30, 220, 50257, 38762, 0, 331, 929, 331, 929, 11, 428, 318, 329, 616, 7627, 7051, 1017, 6606, -100, -100, -100, -100, -100, -100, -100,

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False,
                                                return_tensors="pt")

In [ ]:
training_args = TrainingArguments(
    output_dir                  = "gpt2_pretrain",
    per_device_train_batch_size = 16,    # Start more conservative with quantized model
    per_device_eval_batch_size  = 16,
    gradient_accumulation_steps = 8,    # Still gives effective batch size of 32
    eval_strategy               = "steps",
    save_strategy               = "steps",
    eval_steps                  = 500,
    save_steps                  = 500,
    logging_strategy            = "steps",
    logging_steps               = 100,
    fp16                        = False,  # Fixed: Let BitsAndBytes handle precision
    bf16                        = True,   # Fixed: Let BitsAndBytes handle precision
    report_to                   = [],
    num_train_epochs            = 3,
    load_best_model_at_end      = True,
    metric_for_best_model       = "eval_loss",
    greater_is_better           = False,  # Added: Lower loss is better
    dataloader_pin_memory       = False,  # Added: Recommended for quantized models
    warmup_steps                = 100,    # Added: Helps with training stability
    weight_decay                = 0.01,   # Added: Regularization for better generalization
    save_total_limit            = 2,      # Added: Save space by keeping only 2 best models
    learning_rate               = 2e-4,   # Explicit learning rate
    gradient_checkpointing      = True    # Save memory during training
)

In [ ]:
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_dataset,
    eval_dataset    = val_dataset,
    data_collator   = data_collator,
    callbacks       = [EarlyStoppingCallback(early_stopping_patience=3)],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import torch

# Clear the CUDA cache
torch.cuda.empty_cache()

print("CUDA memory cache cleared.")

CUDA memory cache cleared.


In [ ]:
!pip install peft

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.save_model("gpt2_pretrain/best")
tokenizer.save_pretrained("gpt2_pretrain/best")

base_model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    quantization_config=quantization_config,  # Reuse your original config
    device_map="auto",
    torch_dtype=torch.bfloat16)

# CRITICAL FIX: Load the tokenizer first and add custom tokens to base model
temp_tokenizer = AutoTokenizer.from_pretrained("gpt2_pretrain/best")

# Add the custom tokens to base model to match training setup
if len(temp_tokenizer) > base_model.config.vocab_size:
    print(f"Resizing base model embeddings from {base_model.config.vocab_size} to {len(temp_tokenizer)}")
    base_model.resize_token_embeddings(len(temp_tokenizer))

finetuned_model = PeftModel.from_pretrained(base_model, "gpt2_pretrain/best")
finetuned_tokenizer = AutoTokenizer.from_pretrained("gpt2_pretrain/best")

#finetuned_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Verify your custom tokens loaded correctly
print(f"Loaded tokens - PAD: {finetuned_tokenizer.pad_token}, SEP: {finetuned_tokenizer.sep_token}")

Resizing base model embeddings from 50257 to 50258


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Loaded tokens - PAD: <|endoftext|>, SEP: <SEP>


In [ ]:
def tokenize_for_generation(examples, tokenizer, max_length=30):
    """Tokenization that matches training format for better evaluation"""

    # Match the training format with task prefix
    task_prefix = "Given this rap line, generate the next line: "

    # Create prompts that end with <SEP> (where model should continue)
    inputs = [f"{task_prefix}{line1} {tokenizer.sep_token}" for line1 in examples['line_1']]
    # targets = [f"{line2}{tokenizer.eos_token}" for line2 in examples['line_2']]

    # Tokenize inputs and targets separately
    model_inputs = tokenizer(inputs,
                             padding=True,
                             truncation=True,
                             max_length=max_length,
                             return_tensors="pt")  # Increased for task prefix
    # labels = tokenizer(targets,
    #                    padding=True,
    #                    truncation=True,
    #                    max_length=20,
    #                    return_tensors="pt")

    # model_inputs['labels'] = labels['input_ids']

    # # Mask padding tokens in labels (-100 means ignore in loss calculation)
    # model_inputs['labels'][model_inputs['labels'] == tokenizer.pad_token_id] = -100

    return model_inputs

In [ ]:
# Convert test_pairs to Dataset format
test_examples = {
    'line_1': [pair[0] for pair in test_pairs],
    'line_2': [pair[1] for pair in test_pairs]
}
test_examples = Dataset.from_dict(test_examples)

# Apply your corrected tokenization to test dataset
test_dataset = test_examples.map(
    lambda examples: tokenize_for_generation(examples, tokenizer),
    batched=True,
    remove_columns=['line_1']  # Keep line_2 as reference
)

# Set format for PyTorch
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

print(f"Test dataset tokenized: {len(test_dataset)} examples")
print("Ready for evaluation!")

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Test dataset tokenized: 15000 examples
Ready for evaluation!


In [ ]:
def generate_direct_optimized(first_lines, model, tokenizer, batch_size=128):
    """Direct generation without pipeline - works better with quantized PEFT models"""

    if not first_lines:
        return []

    model.eval()
    generated_lines = []

    task_prefix = 'Given this rap line, generate the next line: '

    # Process in batches for efficiency
    for i in range(0, len(first_lines), batch_size):
        batch_lines = first_lines[i:i + batch_size]

        # Create prompts with separator token
        prompts = [f"{task_prefix}{line} {tokenizer.sep_token}" for line in batch_lines]

        # Tokenize batch
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=30  # Leave room for generation
        ).to(model.device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=20,
                do_sample=True,
                temperature=0.8,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_return_sequences=1
            )

        # Decode generations (remove input prompt)
        for j, output in enumerate(outputs):
            input_length = inputs['input_ids'][j].shape[0]
            generated_tokens = output[input_length:]
            generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            generated_lines.append(generated_text.strip())

    return generated_lines

In [ ]:
# Extract first lines from your test dataset
test_first_lines = [pair[0] for pair in test_pairs]
test_second_lines = [pair[1] for pair in test_pairs]

print(f"Generating {len(test_first_lines)} examples using optimized pipeline...")

# Set padding side to left for decoder-only models (GPT-2)
finetuned_tokenizer.padding_side = "left"

# Generate and store results
generated_lines = generate_direct_optimized(test_first_lines, finetuned_model, finetuned_tokenizer)

test_results = []
for idx in range(len(test_dataset)):
    test_results.append({
        'input': test_pairs[idx][0],
        'actual': test_pairs[idx][1],
        'generated': generated_lines[idx]
    })

# Print first 10 results
print("\nFirst 10 Generation Results:")
print("=" * 60)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Input:    {test_results[i]['input']}")
    print(f"Actual:   {test_results[i]['actual']}")
    print(f"Generated:{test_results[i]['generated']}")
    print("-" * 40)

print(f"\nCompleted generation for all {len(test_results)} examples!")

Generating 15000 examples using optimized pipeline...

First 10 Generation Results:

Example 1:
Input:    and put my head on his  bible
Actual:   i swear to god i didn't do it
Generated:and we'll kill a bitch, just like she did it in the streets of the streets of the
----------------------------------------

Example 2:
Input:    of the picture—punchline, figured out, "ahh, i get you"
Actual:   no, you don't, nigga, so why don't you go'n' figure?
Generated:UCHB  UCHB, yeah, you know that?  UCHB, you know that?
----------------------------------------

Example 3:
Input:    got the world followin' the new york script
Actual:   hustle with timbs and hoodies on my new york flip
Generated:i can't get enough, gettin' too much, gettin' too much, gettin
----------------------------------------

Example 4:
Input:    tell it like it is nigga, tell it like it is homeboy
Actual:   tell it like it is nigga, tell it like it is, tell them what's up!
Generated:it's my homeboy, it's my homeboy, it's my 

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

def calculate_bleu_scores(test_results):
    bleu_scores = []
    smoothing = SmoothingFunction().method1

    for result in test_results:
        reference = [result['actual'].split()]
        candidate = result['generated'].split()

        # Calculate BLEU-4 score
        score = sentence_bleu(reference, candidate, smoothing_function=smoothing)
        bleu_scores.append(score)

    return bleu_scores

# Calculate BLEU scores
bleu_scores = calculate_bleu_scores(test_results)
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"BLEU Score Range: {min(bleu_scores):.4f} - {max(bleu_scores):.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Average BLEU Score: 0.0156
BLEU Score Range: 0.0000 - 0.8242


In [ ]:
from rouge import Rouge
from rouge_score import rouge_scorer

def calculate_rouge_scores(test_results):
    """Calculate ROUGE scores, handling empty generations gracefully"""
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    valid_count = 0
    empty_count = 0

    for result in test_results:
        actual = result['actual'].strip()
        generated = result['generated'].strip()

        # Skip if either actual or generated is empty
        if not actual or not generated:
            empty_count += 1
            # Assign 0 score for empty generations
            rouge1_scores.append(0.0)
            rouge2_scores.append(0.0)
            rougeL_scores.append(0.0)
        else:
            valid_count += 1
            scores = rouge_scorer_obj.score(actual, generated)
            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)

    print(f"ROUGE Calculation: {valid_count} valid, {empty_count} empty generations")

    return {
        'rouge1': {
            'individual_scores': rouge1_scores,
            'average': np.mean(rouge1_scores),
            'std': np.std(rouge1_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        },
        'rouge2': {
            'individual_scores': rouge2_scores,
            'average': np.mean(rouge2_scores),
            'std': np.std(rouge2_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        },
        'rougeL': {
            'individual_scores': rougeL_scores,
            'average': np.mean(rougeL_scores),
            'std': np.std(rougeL_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        }
    }

# Test the fixed function
rouge_scores = calculate_rouge_scores(test_results)

if rouge_scores:
    print("ROUGE Scores:")
    print(f"  ROUGE-1: {rouge_scores['rouge1']['average']:.4f} (±{rouge_scores['rouge1']['std']:.4f})")
    print(f"  ROUGE-2: {rouge_scores['rouge2']['average']:.4f} (±{rouge_scores['rouge2']['std']:.4f})")
    print(f"  ROUGE-L: {rouge_scores['rougeL']['average']:.4f} (±{rouge_scores['rougeL']['std']:.4f})")

ROUGE Calculation: 14996 valid, 4 empty generations
ROUGE Scores:
  ROUGE-1: 0.1027 (±0.1267)
  ROUGE-2: 0.0262 (±0.0942)
  ROUGE-L: 0.0954 (±0.1205)


In [ ]:
# Install BERTScore
!pip install bert-score

from bert_score import score

def calculate_bert_scores(test_results):
    """Calculate BERTScore for all test results"""
    candidates = [result['generated'] for result in test_results]
    references = [result['actual'] for result in test_results]

    # Calculate BERTScore (returns precision, recall, F1)
    P, R, F1 = score(candidates, references, lang="en", verbose=False)

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item(),
        'precision_scores': P.tolist(),
        'recall_scores': R.tolist(),
        'f1_scores': F1.tolist()
    }

# Calculate BERTScore
bert_scores = calculate_bert_scores(test_results)
print(f"BERTScore F1: {bert_scores['f1']:.4f}")
print(f"BERTScore Precision: {bert_scores['precision']:.4f}")
print(f"BERTScore Recall: {bert_scores['recall']:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


BERTScore F1: 0.8262
BERTScore Precision: 0.8197
BERTScore Recall: 0.8331


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dimensional embeddings

def sentence_cosine_similarity(actual_lines, generated_lines):
    """Calculate cosine similarity between whole sentences"""

    # Encode all sentences to get embeddings
    actual_embeddings = model.encode(actual_lines, convert_to_tensor=True)
    generated_embeddings = model.encode(generated_lines, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_scores = util.pytorch_cos_sim(actual_embeddings, generated_embeddings)

    # Extract diagonal (pairwise similarities)
    similarities = [cosine_scores[i][i].item() for i in range(len(actual_lines))]

    return similarities

# Usage with your test results
actual_lines = [result['actual'] for result in test_results]
generated_lines = [result['generated'] for result in test_results]

sentence_similarities = sentence_cosine_similarity(actual_lines, generated_lines)
avg_sentence_similarity = sum(sentence_similarities) / len(sentence_similarities)

print(f"Average Sentence Cosine Similarity: {avg_sentence_similarity:.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Average Sentence Cosine Similarity: 0.2038


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.corpus import cmudict
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import time
from typing import Dict, List, Any
import warnings

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('cmudict', quiet=True)

class ComprehensiveEvaluator:
    def __init__(self, sentence_model_name: str = 'all-MiniLM-L6-v2', device: str = None):
        """
        Initialize evaluator with configurable models and device

        Args:
            sentence_model_name: Name of sentence transformer model to use
            device: Device to run models on ('cuda', 'cpu', or None for auto)
        """
        # Set device
        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        print(f"Initializing evaluator on device: {self.device}")

        # Initialize models with error handling
        try:
            self.sentence_model = SentenceTransformer(sentence_model_name, device=self.device)
            print(f"✓ Loaded Sentence-BERT model: {sentence_model_name}")
        except Exception as e:
            warnings.warn(f"Failed to load Sentence-BERT model: {e}")
            self.sentence_model = None

        # Initialize ROUGE scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        # Initialize CMU dictionary with error handling
        try:
            self.cmu_dict = cmudict.dict()
            print(f"✓ Loaded CMU dictionary with {len(self.cmu_dict)} entries")
        except Exception as e:
            warnings.warn(f"Failed to load CMU dictionary: {e}")
            self.cmu_dict = {}

        # Initialize BLEU smoothing function
        self.bleu_smoothing = SmoothingFunction().method1

        # Cache for performance
        self._rhyme_cache = {}

    def calculate_bleu_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate BLEU scores with improved handling"""
        bleu_scores = []
        valid_count = 0
        empty_count = 0

        for result in test_results:
            actual = result['actual'].strip()
            generated = result['generated'].strip()

            if not generated:
                empty_count += 1
                bleu_scores.append(0.0)
            elif not actual:
                empty_count += 1
                bleu_scores.append(0.0)
            else:
                valid_count += 1
                reference = [actual.split()]
                candidate = generated.split()

                score = sentence_bleu(reference, candidate, smoothing_function=self.bleu_smoothing)
                bleu_scores.append(score)

        return {
            'individual_scores': bleu_scores,
            'average': np.mean(bleu_scores),
            'std': np.std(bleu_scores),
            'min': np.min(bleu_scores),
            'max': np.max(bleu_scores),
            'valid_count': valid_count,
            'empty_count': empty_count
        }

    def calculate_rouge_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate ROUGE scores with enhanced tracking"""
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []

        valid_count = 0
        empty_count = 0

        for result in test_results:
            actual = result['actual'].strip()
            generated = result['generated'].strip()

            if not generated or not actual:
                empty_count += 1
                rouge1_scores.append(0.0)
                rouge2_scores.append(0.0)
                rougeL_scores.append(0.0)
            else:
                valid_count += 1
                scores = self.rouge_scorer.score(actual, generated)
                rouge1_scores.append(scores['rouge1'].fmeasure)
                rouge2_scores.append(scores['rouge2'].fmeasure)
                rougeL_scores.append(scores['rougeL'].fmeasure)

        return {
            'rouge1': {
                'individual_scores': rouge1_scores,
                'average': np.mean(rouge1_scores),
                'std': np.std(rouge1_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            },
            'rouge2': {
                'individual_scores': rouge2_scores,
                'average': np.mean(rouge2_scores),
                'std': np.std(rouge2_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            },
            'rougeL': {
                'individual_scores': rougeL_scores,
                'average': np.mean(rougeL_scores),
                'std': np.std(rougeL_scores),
                'valid_count': valid_count,
                'empty_count': empty_count
            }
        }

    def calculate_bert_scores(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate BERTScore with batch processing"""
        candidates = [result['generated'] for result in test_results]
        references = [result['actual'] for result in test_results]

        try:
            # Calculate BERTScore with device specification
            P, R, F1 = bert_score(candidates, references, lang="en", verbose=False, device=self.device)

            return {
                'precision': {
                    'average': P.mean().item(),
                    'std': P.std().item(),
                    'individual_scores': P.tolist()
                },
                'recall': {
                    'average': R.mean().item(),
                    'std': R.std().item(),
                    'individual_scores': R.tolist()
                },
                'f1': {
                    'average': F1.mean().item(),
                    'std': F1.std().item(),
                    'individual_scores': F1.tolist()
                }
            }
        except Exception as e:
            warnings.warn(f"BERTScore calculation failed: {e}")
            return None

    def calculate_sentence_similarity(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate sentence similarity with error handling"""
        if self.sentence_model is None:
            warnings.warn("Sentence-BERT model not available")
            return None

        actual_lines = [result['actual'] for result in test_results]
        generated_lines = [result['generated'] for result in test_results]

        try:
            # Encode all sentences with batch processing
            actual_embeddings = self.sentence_model.encode(actual_lines, convert_to_tensor=True, show_progress_bar=False)
            generated_embeddings = self.sentence_model.encode(generated_lines, convert_to_tensor=True, show_progress_bar=False)

            # Calculate cosine similarity
            cosine_scores = util.pytorch_cos_sim(actual_embeddings, generated_embeddings)

            # Extract diagonal (pairwise similarities)
            similarities = [cosine_scores[i][i].item() for i in range(len(actual_lines))]

            return {
                'individual_scores': similarities,
                'average': np.mean(similarities),
                'std': np.std(similarities),
                'min': np.min(similarities),
                'max': np.max(similarities)
            }
        except Exception as e:
            warnings.warn(f"Sentence similarity calculation failed: {e}")
            return None

    def get_last_word(self, line: str) -> str:
        """Extract last word with improved cleaning"""
        import re
        # Use regex to better handle punctuation and contractions
        words = re.findall(r"\b[a-zA-Z]+(?:'[a-zA-Z]+)?\b", line.lower())
        return words[-1] if words else ""

    def get_rhyme_part_cmu(self, word: str) -> List[str]:
        """Extract rhyming part with caching"""
        if word in self._rhyme_cache:
            return self._rhyme_cache[word]

        if word in self.cmu_dict:
            pronunciations = self.cmu_dict[word]
            if pronunciations:
                # Get the part after the last stressed vowel
                pron = pronunciations[0]
                for i in range(len(pron) - 1, -1, -1):
                    if pron[i][-1].isdigit():  # Stressed vowel
                        result = pron[i:]
                        self._rhyme_cache[word] = result
                        return result

        self._rhyme_cache[word] = None
        return None

    def analyze_rhymes_cmu(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Enhanced rhyme analysis with better statistics"""
        phonetic_rhymes = 0
        near_rhymes = 0  # Rhymes with similar endings
        total_valid = 0
        total_processed = 0

        rhyme_details = []

        for i, result in enumerate(test_results):
            input_last = self.get_last_word(result['input'])
            generated_last = self.get_last_word(result['generated'])

            total_processed += 1

            if input_last and generated_last:
                input_rhyme = self.get_rhyme_part_cmu(input_last)
                generated_rhyme = self.get_rhyme_part_cmu(generated_last)

                if input_rhyme and generated_rhyme:
                    total_valid += 1

                    is_perfect_rhyme = input_rhyme == generated_rhyme
                    is_near_rhyme = False

                    # Check for near rhymes (last 2 phonemes match)
                    if not is_perfect_rhyme and len(input_rhyme) >= 2 and len(generated_rhyme) >= 2:
                        is_near_rhyme = input_rhyme[-2:] == generated_rhyme[-2:]

                    if is_perfect_rhyme:
                        phonetic_rhymes += 1
                    elif is_near_rhyme:
                        near_rhymes += 1

                    rhyme_details.append({
                        'example_index': i,
                        'input_word': input_last,
                        'generated_word': generated_last,
                        'input_phonemes': input_rhyme,
                        'generated_phonemes': generated_rhyme,
                        'is_perfect_rhyme': is_perfect_rhyme,
                        'is_near_rhyme': is_near_rhyme
                    })

        return {
            'perfect_rhyme_rate': phonetic_rhymes / total_valid if total_valid > 0 else 0,
            'near_rhyme_rate': near_rhymes / total_valid if total_valid > 0 else 0,
            'total_rhyme_rate': (phonetic_rhymes + near_rhymes) / total_valid if total_valid > 0 else 0,
            'perfect_rhymes': phonetic_rhymes,
            'near_rhymes': near_rhymes,
            'total_valid': total_valid,
            'total_processed': total_processed,
            'coverage': total_valid / total_processed if total_processed > 0 else 0,
            'details': rhyme_details
        }

    def calculate_additional_metrics(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Calculate additional rap-specific metrics"""

        # Syllable analysis (approximate)
        def count_syllables(word):
            # Simple syllable counting heuristic
            word = word.lower()
            count = 0
            vowels = "aeiouy"
            if word[0] in vowels:
                count += 1
            for i in range(1, len(word)):
                if word[i] in vowels and word[i-1] not in vowels:
                    count += 1
            if word.endswith("e"):
                count -= 1
            if count == 0:
                count += 1
            return count

        syllable_diffs = []
        word_diversity_scores = []

        for result in test_results:
            actual_words = result['actual'].split()
            generated_words = result['generated'].split()

            # Syllable analysis
            if actual_words and generated_words:
                actual_syllables = sum(count_syllables(word) for word in actual_words)
                generated_syllables = sum(count_syllables(word) for word in generated_words)
                syllable_diffs.append(abs(actual_syllables - generated_syllables))
            else:
                syllable_diffs.append(0)

            # Word diversity (unique words / total words)
            if generated_words:
                diversity = len(set(generated_words)) / len(generated_words)
                word_diversity_scores.append(diversity)
            else:
                word_diversity_scores.append(0.0)

        return {
            'syllable_similarity': {
                'average_diff': np.mean(syllable_diffs),
                'std_diff': np.std(syllable_diffs)
            },
            'word_diversity': {
                'average': np.mean(word_diversity_scores),
                'std': np.std(word_diversity_scores)
            }
        }

    def evaluate_comprehensive(self, test_results: List[Dict], show_progress: bool = True) -> Dict[str, Any]:
        """Enhanced comprehensive evaluation with timing and progress"""
        start_time = time.time()

        print("=" * 80)
        print("COMPREHENSIVE EVALUATION RESULTS")
        print("=" * 80)

        # Basic statistics
        total_examples = len(test_results)
        empty_generations = sum(1 for r in test_results if not r['generated'].strip())

        print(f"Dataset Statistics:")
        print(f"  Total Examples: {total_examples}")
        print(f"  Empty Generations: {empty_generations} ({empty_generations/total_examples:.1%})")
        print()

        # Calculate all metrics with timing
        results = {}

        if show_progress:
            print("Computing metrics...")

        # Traditional NLP metrics
        if show_progress: print("  • BLEU scores...")
        results['bleu'] = self.calculate_bleu_scores(test_results)

        if show_progress: print("  • ROUGE scores...")
        results['rouge'] = self.calculate_rouge_scores(test_results)

        if show_progress: print("  • BERTScore...")
        results['bert_score'] = self.calculate_bert_scores(test_results)

        # Sentence-level similarity
        if show_progress: print("  • Sentence similarity...")
        results['sentence_similarity'] = self.calculate_sentence_similarity(test_results)

        # Rhyme analysis
        if show_progress: print("  • Rhyme analysis...")
        results['cmu_rhyme'] = self.analyze_rhymes_cmu(test_results)

        # Additional metrics
        if show_progress: print("  • Additional metrics...")
        results['additional_metrics'] = self.calculate_additional_metrics(test_results)

        # Length analysis
        results['length_analysis'] = self.calculate_length_similarity(test_results)

        # Display results with enhanced formatting
        self._display_results(results, total_examples, empty_generations)

        execution_time = time.time() - start_time
        print(f"\nEvaluation completed in {execution_time:.2f} seconds")
        print("=" * 80)

        # Add metadata
        results['metadata'] = {
            'total_examples': total_examples,
            'empty_generations': empty_generations,
            'execution_time': execution_time,
            'device_used': self.device
        }

        return results

    def _display_results(self, results: Dict, total_examples: int, empty_generations: int):
        """Enhanced result display with better formatting"""

        print("\n" + "="*60)
        print("TRADITIONAL NLP METRICS")
        print("="*60)

        # BLEU
        bleu = results['bleu']
        print(f"BLEU Score:")
        print(f"  Average: {bleu['average']:.4f} (±{bleu['std']:.4f})")
        print(f"  Range: {bleu['min']:.4f} - {bleu['max']:.4f}")
        print(f"  Valid/Empty: {bleu['valid_count']}/{bleu['empty_count']}")

        # ROUGE
        rouge = results['rouge']
        print(f"\nROUGE Scores:")
        print(f"  ROUGE-1: {rouge['rouge1']['average']:.4f} (±{rouge['rouge1']['std']:.4f})")
        print(f"  ROUGE-2: {rouge['rouge2']['average']:.4f} (±{rouge['rouge2']['std']:.4f})")
        print(f"  ROUGE-L: {rouge['rougeL']['average']:.4f} (±{rouge['rougeL']['std']:.4f})")
        print(f"  Valid/Empty: {rouge['rouge1']['valid_count']}/{rouge['rouge1']['empty_count']}")

        # BERTScore
        if results['bert_score']:
            bert = results['bert_score']
            print(f"\nBERTScore:")
            print(f"  F1: {bert['f1']['average']:.4f} (±{bert['f1']['std']:.4f})")
            print(f"  Precision: {bert['precision']['average']:.4f} (±{bert['precision']['std']:.4f})")
            print(f"  Recall: {bert['recall']['average']:.4f} (±{bert['recall']['std']:.4f})")

        # Sentence similarity
        if results['sentence_similarity']:
            sent_sim = results['sentence_similarity']
            print("\n" + "="*60)
            print("SENTENCE-LEVEL SEMANTIC SIMILARITY")
            print("="*60)
            print(f"Sentence-BERT Cosine Similarity:")
            print(f"  Average: {sent_sim['average']:.4f} (±{sent_sim['std']:.4f})")
            print(f"  Range: {sent_sim['min']:.4f} - {sent_sim['max']:.4f}")

        # Rhyme analysis
        rhyme = results['cmu_rhyme']
        print("\n" + "="*60)
        print("RHYME ANALYSIS")
        print("="*60)
        print(f"CMU Dictionary Phonetic Analysis:")
        print(f"  Perfect Rhyme Rate: {rhyme['perfect_rhyme_rate']:.2%}")
        print(f"  Near Rhyme Rate: {rhyme['near_rhyme_rate']:.2%}")
        print(f"  Total Rhyme Rate: {rhyme['total_rhyme_rate']:.2%}")
        print(f"  Dictionary Coverage: {rhyme['coverage']:.1%} ({rhyme['total_valid']}/{rhyme['total_processed']})")

        # Additional metrics
        additional = results['additional_metrics']
        print("\n" + "="*60)
        print("RAP-SPECIFIC METRICS")
        print("="*60)
        print(f"Syllable Similarity:")
        print(f"  Average Difference: {additional['syllable_similarity']['average_diff']:.2f} syllables")
        print(f"Word Diversity:")
        print(f"  Average: {additional['word_diversity']['average']:.3f}")

        # Length analysis
        length = results['length_analysis']
        print(f"\nLength Analysis:")
        print(f"  Average Length Difference: {length['average_length_diff']:.2f} words")
        print(f"  Average Length Ratio: {length['average_length_ratio']:.2f}")

    def calculate_length_similarity(self, test_results: List[Dict]) -> Dict[str, Any]:
        """Enhanced length analysis"""
        length_diffs = []
        length_ratios = []

        for result in test_results:
            actual_len = len(result['actual'].split())
            generated_len = len(result['generated'].split())

            length_diffs.append(abs(actual_len - generated_len))

            if actual_len > 0:
                length_ratios.append(generated_len / actual_len)
            else:
                length_ratios.append(0.0)

        return {
            'average_length_diff': np.mean(length_diffs),
            'std_length_diff': np.std(length_diffs),
            'average_length_ratio': np.mean(length_ratios),
            'std_length_ratio': np.std(length_ratios),
            'individual_diffs': length_diffs,
            'individual_ratios': length_ratios
        }

    def analyze_top_examples(self, results: Dict, test_results: List[Dict], top_n: int = 5):
      """Find and display best performing examples"""
      # Get BERTScore F1 scores
      if results['bert_score']:
          f1_scores = results['bert_score']['f1']['individual_scores']

          # Find top examples
          top_indices = np.argsort(f1_scores)[-top_n:][::-1]

          print(f"\nTop {top_n} Generated Examples (by BERTScore F1):")
          print("-" * 60)

          for i, idx in enumerate(top_indices, 1):
              result = test_results[idx]
              score = f1_scores[idx]
              print(f"\nExample {i} (BERTScore F1: {score:.4f}):")
              print(f"Input:     {result['input']}")
              print(f"Actual:    {result['actual']}")
              print(f"Generated: {result['generated']}")

          return top_indices

In [ ]:
# Initialize with custom settings
evaluator = ComprehensiveEvaluator(
    sentence_model_name='all-MiniLM-L6-v2',  # or 'all-mpnet-base-v2' for better quality
    device='cuda'  # or 'cpu'
)

# Run evaluation
comprehensive_results = evaluator.evaluate_comprehensive(test_results, show_progress=True)
evaluator.analyze_top_examples(comprehensive_results, test_results)

Initializing evaluator on device: cuda
✓ Loaded Sentence-BERT model: all-MiniLM-L6-v2
✓ Loaded CMU dictionary with 123455 entries
COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 4 (0.0%)

Computing metrics...
  • BLEU scores...
  • ROUGE scores...
  • BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  • Sentence similarity...
  • Rhyme analysis...
  • Additional metrics...

TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0156 (±0.0554)
  Range: 0.0000 - 0.8242
  Valid/Empty: 14996/4

ROUGE Scores:
  ROUGE-1: 0.1027 (±0.1267)
  ROUGE-2: 0.0262 (±0.0942)
  ROUGE-L: 0.0954 (±0.1205)
  Valid/Empty: 14996/4

BERTScore:
  F1: 0.8262 (±0.0292)
  Precision: 0.8197 (±0.0313)
  Recall: 0.8331 (±0.0320)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.2038 (±0.1495)
  Range: -0.1330 - 0.9934

RHYME ANALYSIS
CMU Dictionary Phonetic Analysis:
  Perfect Rhyme Rate: 12.65%
  Near Rhyme Rate: 0.05%
  Total Rhyme Rate: 12.71%
  Dictionary Coverage: 84.9% (12731/15000)

RAP-SPECIFIC METRICS
Syllable Similarity:
  Average Difference: 6.34 syllables
Word Diversity:
  Average: 0.724

Length Analysis:
  Average Length Difference: 6.11 words
  Average Length Ratio: 2.14

Evaluation completed in 67.46 seconds

Top 5 Generated Examples (by BERTScore F1):
-----------------

array([ 9721,   810,  4174, 13293,  1729])

In [ ]:
### Finetuning with Lora

In [ ]:
modela_lora = finetuned_model

Resized embeddings to 50258 tokens
trainable params: 2,359,296 || all params: 126,799,872 || trainable%: 1.8606


In [ ]:
# LoRA training arguments (can be more aggressive)
lora_training_args = TrainingArguments(
    output_dir="gpt2_pretrain_finetune_lora",
    per_device_train_batch_size=16,    # Larger batch size possible
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=3,               # Can train longer
    learning_rate=1e-4,               # Higher LR works well with LoRA
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    fp16=False,
    bf16=True,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    save_total_limit=2
)

In [ ]:
# Create LoRA trainer
lora_trainer = Trainer(
    model=model_lora,
    args=lora_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train with LoRA
print("Starting LoRA fine-tuning...")
lora_trainer.train()
lora_trainer.save_model("gpt2_pretrain_finetune_lora/best")

model_lora.save_pretrained("gpt2_pretrain_finetune_lora/best")
finetuned_tokenizer.save_pretrained("gpt2_pretrain_finetune_lora/best")

finetuned_model_lora = PeftModel.from_pretrained(finetuned_model, "gpt2_pretrain_finetune_lora/best")
finetuned_tokenizer_lora = AutoTokenizer.from_pretrained("gpt2_pretrain_finetune_lora/best")
# finetuned_model_lora.to("cuda" if torch.cuda.is_available() else "cpu")

Starting LoRA fine-tuning...


Step,Training Loss,Validation Loss
500,3.057000,3.103932
1000,3.042400,3.086456


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters i

In [ ]:
# Extract first lines from your test dataset
test_first_lines = [pair[0] for pair in test_pairs]
test_second_lines = [pair[1] for pair in test_pairs]

print(f"Generating {len(test_first_lines)} examples using optimized pipeline...")

# Set padding side to left for decoder-only models (GPT-2)
finetuned_tokenizer_lora.padding_side = "left"

# Generate and store results
generated_lines = generate_direct_optimized(test_first_lines, finetuned_model_lora, finetuned_tokenizer_lora)

test_results = []
for idx in range(len(test_dataset)):
    test_results.append({
        'input': test_pairs[idx][0],
        'actual': test_pairs[idx][1],
        'generated': generated_lines[idx]
    })

# Print first 10 results
print("\nFirst 10 Generation Results:")
print("=" * 60)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Input:    {test_results[i]['input']}")
    print(f"Actual:   {test_results[i]['actual']}")
    print(f"Generated:{test_results[i]['generated']}")
    print("-" * 40)

print(f"\nCompleted generation for all {len(test_results)} examples!")

Generating 15000 examples using optimized pipeline...

First 10 Generation Results:

Example 1:
Input:    and put my head on his  bible
Actual:   i swear to god i didn't do it
Generated:and put my mouth on his bible, i'm a genius, i can't stop it, i
----------------------------------------

Example 2:
Input:    of the picture—punchline, figured out, "ahh, i get you"
Actual:   no, you don't, nigga, so why don't you go'n' figure?
Generated:i don't see that, i'm not a bad rapper, but you know that, and
----------------------------------------

Example 3:
Input:    got the world followin' the new york script
Actual:   hustle with timbs and hoodies on my new york flip
Generated:i'm still got the york script, i'm still got the york script, i'm
----------------------------------------

Example 4:
Input:    tell it like it is nigga, tell it like it is homeboy
Actual:   tell it like it is nigga, tell it like it is, tell them what's up!
Generated:it's like it's homeboy, it's like it's homeboy, i

In [ ]:
# Initialize with custom settings
evaluator = ComprehensiveEvaluator(
    sentence_model_name='all-MiniLM-L6-v2',  # or 'all-mpnet-base-v2' for better quality
    device='cuda'  # or 'cpu'
)

# Run evaluation
comprehensive_results = evaluator.evaluate_comprehensive(test_results, show_progress=True)

evaluator.analyze_top_examples(comprehensive_results, test_results)

Initializing evaluator on device: cuda
✓ Loaded Sentence-BERT model: all-MiniLM-L6-v2
✓ Loaded CMU dictionary with 123455 entries
COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 0 (0.0%)

Computing metrics...
  • BLEU scores...
  • ROUGE scores...
  • BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  • Sentence similarity...
  • Rhyme analysis...
  • Additional metrics...

TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0149 (±0.0516)
  Range: 0.0000 - 0.9122
  Valid/Empty: 15000/0

ROUGE Scores:
  ROUGE-1: 0.1025 (±0.1236)
  ROUGE-2: 0.0251 (±0.0894)
  ROUGE-L: 0.0948 (±0.1166)
  Valid/Empty: 15000/0

BERTScore:
  F1: 0.8267 (±0.0253)
  Precision: 0.8208 (±0.0277)
  Recall: 0.8331 (±0.0283)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.2049 (±0.1465)
  Range: -0.0902 - 0.9953

RHYME ANALYSIS
CMU Dictionary Phonetic Analysis:
  Perfect Rhyme Rate: 11.04%
  Near Rhyme Rate: 0.07%
  Total Rhyme Rate: 11.11%
  Dictionary Coverage: 85.0% (12747/15000)

RAP-SPECIFIC METRICS
Syllable Similarity:
  Average Difference: 6.43 syllables
Word Diversity:
  Average: 0.723

Length Analysis:
  Average Length Difference: 6.25 words
  Average Length Ratio: 2.18

Evaluation completed in 66.67 seconds

Top 5 Generated Examples (by BERTScore F1):
-----------------

array([ 3744,  2611,  4174, 13293,  1729])

In [ ]:
### Finetuning with attention layer only

In [ ]:
def freeze_all_except_attention(model):
    """Freeze everything except attention mechanisms and layer norms"""

    # Access the base model through PEFT wrapper
    if hasattr(model, 'base_model'):
        base_model = model.base_model.model  # Navigate PEFT structure
    else:
        base_model = model

    print(f"Working with model type: {type(model)}")
    print(f"Base model type: {type(base_model)}")

    # Freeze all parameters first
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze attention layers and related components
    try:
        for i, layer in enumerate(base_model.transformer.h):
            # Unfreeze attention weights (most important for rap patterns)
            if hasattr(layer, 'attn'):
                for name, param in layer.attn.named_parameters():
                    if param.dtype != torch.uint8:  # Skip quantized weights
                        param.requires_grad = True
                        print(f"✅ Unfroze attention layer {i}: {name}")

            # Keep layer norms unfrozen (they work closely with attention)
            if hasattr(layer, 'ln_1'):
                for name, param in layer.ln_1.named_parameters():
                    if param.dtype != torch.uint8:
                        param.requires_grad = True
                        print(f"✅ Unfroze layer norm {i}.ln_1: {name}")

            if hasattr(layer, 'ln_2'):
                for name, param in layer.ln_2.named_parameters():
                    if param.dtype != torch.uint8:
                        param.requires_grad = True
                        print(f"✅ Unfroze layer norm {i}.ln_2: {name}")

        # Keep final layer norm and LM head unfrozen
        if hasattr(base_model.transformer, 'ln_f'):
            for name, param in base_model.transformer.ln_f.named_parameters():
                if param.dtype != torch.uint8:
                    param.requires_grad = True
                    print(f"✅ Unfroze final layer norm: {name}")

        if hasattr(base_model, 'lm_head'):
            for name, param in base_model.lm_head.named_parameters():
                if param.dtype != torch.uint8:
                    param.requires_grad = True
                    print(f"✅ Unfroze LM head: {name}")

    except Exception as e:
        print(f"⚠️ Error accessing model layers: {e}")
        print("Falling back to parameter name-based approach...")

        # Fallback: Use parameter names to identify attention components
        for name, param in model.named_parameters():
            if any(keyword in name.lower() for keyword in ['attn', 'attention', 'ln_1', 'ln_2', 'ln_f', 'lm_head']):
                if param.dtype != torch.uint8:  # Skip quantized parameters
                    param.requires_grad = True
                    print(f"✅ Unfroze: {name}")

    # Count trainable parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\nAttention-only fine-tuning setup:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Trainable percentage: {100 * trainable_params / total_params:.2f}%")

    return model

# Apply attention-specific setup
model_attn = freeze_all_except_attention(finetuned_model)

Working with model type: <class 'peft.peft_model.PeftModelForCausalLM'>
Base model type: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
✅ Unfroze attention layer 0: c_attn.base_layer.bias
✅ Unfroze attention layer 0: c_attn.lora_A.default.weight
✅ Unfroze attention layer 0: c_attn.lora_B.default.weight
✅ Unfroze attention layer 0: c_proj.base_layer.bias
✅ Unfroze attention layer 0: c_proj.lora_A.default.weight
✅ Unfroze attention layer 0: c_proj.lora_B.default.weight
✅ Unfroze layer norm 0.ln_1: weight
✅ Unfroze layer norm 0.ln_1: bias
✅ Unfroze layer norm 0.ln_2: weight
✅ Unfroze layer norm 0.ln_2: bias
✅ Unfroze attention layer 1: c_attn.base_layer.bias
✅ Unfroze attention layer 1: c_attn.lora_A.default.weight
✅ Unfroze attention layer 1: c_attn.lora_B.default.weight
✅ Unfroze attention layer 1: c_proj.base_layer.bias
✅ Unfroze attention layer 1: c_proj.lora_A.default.weight
✅ Unfroze attention layer 1: c_proj.lora_B.default.weight
✅ Unfroze layer norm 1.ln_1: weigh

In [ ]:
# Attention-focused training arguments
attention_training_args = TrainingArguments(
    output_dir="gpt2_pretrain_finetune_attn",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=3,               # Can train longer since fewer parameters
    learning_rate=1e-4,               # Moderate LR for attention layers
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    save_strategy="steps",
    warmup_steps=500,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    fp16=False,
    bf16=True,
    dataloader_pin_memory=False,
    save_total_limit=2
)

In [ ]:
# Create attention trainer
attention_trainer = Trainer(
    model=model_attn,
    args=attention_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train attention-only
print("Starting attention-specific fine-tuning...")
attention_trainer.train()
attention_trainer.save_model("gpt2_pretrain_finetune_attn/best")

model_attn.save_pretrained("gpt2_pretrain_finetune_attn/best")
finetuned_tokenizer.save_pretrained("gpt2_pretrain_finetune_attn/best")

finetuned_model_attention = model_attn  # Use the already trained model
finetuned_tokenizer_attention = finetuned_tokenizer  # Use existing tokenizer

In [ ]:
# Extract first lines from your test dataset
test_first_lines = [pair[0] for pair in test_pairs]
test_second_lines = [pair[1] for pair in test_pairs]

print(f"Generating {len(test_first_lines)} examples using optimized pipeline...")

# Set padding side to left for decoder-only models (GPT-2)
finetuned_tokenizer_attention.padding_side = "left"

# Generate and store results
generated_lines = generate_direct_optimized(test_first_lines, finetuned_model_attention, finetuned_tokenizer_attention)

test_results = []
for idx in range(len(test_dataset)):
    test_results.append({
        'input': test_pairs[idx][0],
        'actual': test_pairs[idx][1],
        'generated': generated_lines[idx]
    })

# Print first 10 results
print("\nFirst 10 Generation Results:")
print("=" * 60)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Input:    {test_results[i]['input']}")
    print(f"Actual:   {test_results[i]['actual']}")
    print(f"Generated:{test_results[i]['generated']}")
    print("-" * 40)

print(f"\nCompleted generation for all {len(test_results)} examples!")

Generating 15000 examples using optimized pipeline...

First 10 Generation Results:

Example 1:
Input:    and put my head on his  bible
Actual:   i swear to god i didn't do it
Generated:my neck on his bible, my neck on his bible, my neck on his bible, my neck
----------------------------------------

Example 2:
Input:    of the picture—punchline, figured out, "ahh, i get you"
Actual:   no, you don't, nigga, so why don't you go'n' figure?
Generated:i'm in the street, and i'm on the beat, you gotta be on the beat
----------------------------------------

Example 3:
Input:    got the world followin' the new york script
Actual:   hustle with timbs and hoodies on my new york flip
Generated:i'm going to get the city, i'm in a hurry, i'm with the gangsters
----------------------------------------

Example 4:
Input:    tell it like it is nigga, tell it like it is homeboy
Actual:   tell it like it is nigga, tell it like it is, tell them what's up!
Generated:tell it like it is homeboy, tell it l

In [ ]:
# Initialize with custom settings
evaluator = ComprehensiveEvaluator(
    sentence_model_name='all-MiniLM-L6-v2',  # or 'all-mpnet-base-v2' for better quality
    device='cuda'  # or 'cpu'
)

# Run evaluation
comprehensive_results = evaluator.evaluate_comprehensive(test_results, show_progress=True)

evaluator.analyze_top_examples(comprehensive_results, test_results)

Initializing evaluator on device: cuda
✓ Loaded Sentence-BERT model: all-MiniLM-L6-v2
✓ Loaded CMU dictionary with 123455 entries
COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 0 (0.0%)

Computing metrics...
  • BLEU scores...
  • ROUGE scores...
  • BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  • Sentence similarity...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  • Rhyme analysis...
  • Additional metrics...

TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0147 (±0.0513)
  Range: 0.0000 - 1.0000
  Valid/Empty: 15000/0

ROUGE Scores:
  ROUGE-1: 0.1019 (±0.1235)
  ROUGE-2: 0.0248 (±0.0903)
  ROUGE-L: 0.0946 (±0.1171)
  Valid/Empty: 15000/0

BERTScore:
  F1: 0.8267 (±0.0253)
  Precision: 0.8208 (±0.0279)
  Recall: 0.8330 (±0.0282)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.2044 (±0.1453)
  Range: -0.1420 - 1.0000

RHYME ANALYSIS
CMU Dictionary Phonetic Analysis:
  Perfect Rhyme Rate: 11.63%
  Near Rhyme Rate: 0.08%
  Total Rhyme Rate: 11.71%
  Dictionary Coverage: 85.0% (12753/15000)

RAP-SPECIFIC METRICS
Syllable Similarity:
  Average Difference: 6.44 syllables
Word Diversity:
  Average: 0.720

Length Analysis:
  Average Length Difference: 6.25 words
  Average Length Ratio: 2.18

Evaluation completed in 66.99 seconds

Top 5 Generated Examples (by BERTScore F1):
--------------------------------------------

array([ 7428,  9050,  4174, 13293,  1904])

In [ ]:
### Finetune with both Lora and attention

In [ ]:
# Configure LoRA specifically for attention mechanisms
attention_lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=24,                    # Slightly higher rank for attention focus
    lora_alpha=48,           # 2x the rank
    lora_dropout=0.05,       # Lower dropout for attention
    target_modules=[         # Focus ONLY on attention components
        "c_attn",           # The key attention mechanism
        "c_proj"            # Attention output projection
    ],
    bias="none"
)

# Handle existing PEFT model properly
if hasattr(finetuned_model, 'peft_config'):
    print("Merging existing PEFT adapters for clean LoRA application...")
    base_for_lora = finetuned_model.merge_and_unload()

    # Re-prepare for quantized training
    base_for_lora.gradient_checkpointing_enable()
    base_for_lora = prepare_model_for_kbit_training(base_for_lora)
else:
    base_for_lora = finetuned_model

# Apply attention-focused LoRA
model_attention_lora = get_peft_model(base_for_lora, attention_lora_config)
model_attention_lora.print_trainable_parameters()

Merging existing PEFT adapters for clean LoRA application...
trainable params: 2,433,024 || all params: 126,873,600 || trainable%: 1.9177


In [ ]:
# Hybrid training arguments
hybrid_training_args = TrainingArguments(
    output_dir="gpt2_pretrain_finetune_hybrid",
    per_device_train_batch_size=16,   # Even larger batch possible
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=3,               # Can train longer
    learning_rate=2e-4,             # Higher LR for LoRA on attention
    weight_decay=0.01,               # Lower weight decay
    lr_scheduler_type="cosine",
    warmup_steps=500,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    fp16=False,
    bf16=True,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    save_total_limit=2
)

In [ ]:
# Train hybrid approach
hybrid_trainer = Trainer(
    model=model_attention_lora,
    args=hybrid_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("Starting hybrid LoRA + Attention fine-tuning...")
hybrid_trainer.train()
hybrid_trainer.save_model("gpt2_pretrain_finetune_hybrid/best")
finetuned_tokenizer.save_pretrained("gpt2_pretrain_finetune_hybrid/best")

finetuned_model_hybrid = model_attention_lora  # Use the already trained model
finetuned_tokenizer_hybrid = finetuned_tokenizer  # Use existing tokenizer

Starting hybrid LoRA + Attention fine-tuning...


Step,Training Loss,Validation Loss
500,2.785500,2.807243
1000,2.736900,2.764750


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:

# Extract first lines from your test dataset
test_first_lines = [pair[0] for pair in test_pairs]
test_second_lines = [pair[1] for pair in test_pairs]

print(f"Generating {len(test_first_lines)} examples using optimized pipeline...")

# Set padding side to left for decoder-only models (GPT-2)
finetuned_tokenizer_hybrid.padding_side = "left"

# Generate and store results
generated_lines = generate_direct_optimized(test_first_lines, finetuned_model_hybrid, finetuned_tokenizer_hybrid)

test_results = []
for idx in range(len(test_dataset)):
    test_results.append({
        'input': test_pairs[idx][0],
        'actual': test_pairs[idx][1],
        'generated': generated_lines[idx]
    })

# Print first 10 results
print("\nFirst 10 Generation Results:")
print("=" * 60)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Input:    {test_results[i]['input']}")
    print(f"Actual:   {test_results[i]['actual']}")
    print(f"Generated:{test_results[i]['generated']}")
    print("-" * 40)

print(f"\nCompleted generation for all {len(test_results)} examples!")

Generating 15000 examples using optimized pipeline...

First 10 Generation Results:

Example 1:
Input:    and put my head on his  bible
Actual:   i swear to god i didn't do it
Generated:a fuckin' diamond, and the other one's a bitch, and my head on the bible and
----------------------------------------

Example 2:
Input:    of the picture—punchline, figured out, "ahh, i get you"
Actual:   no, you don't, nigga, so why don't you go'n' figure?
Generated:she was the one that told me, "hey, you just got to get it" (
----------------------------------------

Example 3:
Input:    got the world followin' the new york script
Actual:   hustle with timbs and hoodies on my new york flip
Generated:got the world followin' the new york script, got the world followin' the new
----------------------------------------

Example 4:
Input:    tell it like it is nigga, tell it like it is homeboy
Actual:   tell it like it is nigga, tell it like it is, tell them what's up!
Generated:tell it like it is, tell i

In [ ]:
# Initialize with custom settings
evaluator = ComprehensiveEvaluator(
    sentence_model_name='all-MiniLM-L6-v2',  # or 'all-mpnet-base-v2' for better quality
    device='cuda'  # or 'cpu'
)

# Run evaluation
comprehensive_results = evaluator.evaluate_comprehensive(test_results, show_progress=True)

evaluator.analyze_top_examples(comprehensive_results, test_results)

Initializing evaluator on device: cuda
✓ Loaded Sentence-BERT model: all-MiniLM-L6-v2
✓ Loaded CMU dictionary with 123455 entries
COMPREHENSIVE EVALUATION RESULTS
Dataset Statistics:
  Total Examples: 15000
  Empty Generations: 0 (0.0%)

Computing metrics...
  • BLEU scores...
  • ROUGE scores...
  • BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  • Sentence similarity...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  • Rhyme analysis...
  • Additional metrics...

TRADITIONAL NLP METRICS
BLEU Score:
  Average: 0.0158 (±0.0572)
  Range: 0.0000 - 1.0000
  Valid/Empty: 15000/0

ROUGE Scores:
  ROUGE-1: 0.1035 (±0.1271)
  ROUGE-2: 0.0262 (±0.0953)
  ROUGE-L: 0.0967 (±0.1215)
  Valid/Empty: 15000/0

BERTScore:
  F1: 0.8268 (±0.0259)
  Precision: 0.8207 (±0.0283)
  Recall: 0.8334 (±0.0289)

SENTENCE-LEVEL SEMANTIC SIMILARITY
Sentence-BERT Cosine Similarity:
  Average: 0.2065 (±0.1507)
  Range: -0.1547 - 1.0000

RHYME ANALYSIS
CMU Dictionary Phonetic Analysis:
  Perfect Rhyme Rate: 13.12%
  Near Rhyme Rate: 0.08%
  Total Rhyme Rate: 13.20%
  Dictionary Coverage: 85.2% (12773/15000)

RAP-SPECIFIC METRICS
Syllable Similarity:
  Average Difference: 6.28 syllables
Word Diversity:
  Average: 0.712

Length Analysis:
  Average Length Difference: 6.08 words
  Average Length Ratio: 2.15

Evaluation completed in 67.06 seconds

Top 5 Generated Examples (by BERTScore F1):
--------------------------------------------

array([7428, 4174, 1904, 9294, 1729])